<a href="https://colab.research.google.com/github/gek7537/Basic_Python/blob/master/%5B%EC%8B%A4%EC%8A%B5_2%5D%20Brain_classification/Brain_classification_CJA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [61]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#시드고정
torch.manual_seed(1)
np.random.seed(1)

#파라미터
learning_rate = 0.0001
nb_epochs = 1000

#데이터로드 및 셔플
al_data = pd.read_excel('/content/drive/My Drive/인공지능 두뇌지수 데이터_1024.xlsx', usecols=['Scholarly Output','Most recent publication','Citations','Citations per Publication','Field-Weighted Citation Impact','h-index','Country Number','Scholarly Output100','Citations per Publication100','Field-Weighted Citation Impact100','Degree'])
xy_data = al_data.values

np.random.shuffle(xy_data)

#print(al_data)
#print(xy_data)

#데이터 구분
x_data = xy_data[:,0:-1]
y_data = xy_data[:,[-1]]

#데이터 텐서로 변환
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

x_test = torch.FloatTensor(x_data)
y_test = torch.FloatTensor(y_data)


#x 데이터 값 정규화
mu = x_train.mean(dim=0)
sigma = x_train.std(dim=0)
norm_x_train = (x_train - mu) / sigma

#x 테스트 값 정규화
mu = x_test.mean(dim=0)
sigma = x_test.std(dim=0)
norm_x_test = (x_train - mu) / sigma

#print(norm_x_train)
print(norm_x_train.shape)
print(y_train.shape)

#x_data.dtype
y_data.dtype


torch.Size([12500, 10])
torch.Size([12500, 1])


dtype('float64')

In [63]:
#검증데이터 테스트데이터 구분




#모델생성

linear1 = torch.nn.Linear(10, 256, bias=True)
linear2 = torch.nn.Linear(256, 1, bias=True)
relu = torch.nn.ReLU()

model = torch.nn.Sequential(linear1, relu, linear2)
#print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(nb_epochs + 1):
    hypothesis = model(x_train)


    #cost = F.cross_entropy(hypothesis, y_train)
    #cost = torch.nn.CrossEntropyLoss(hypothesis, y_train)
    cost = F.mse_loss(hypothesis, y_train)


    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 14190.850586
Epoch  100/1000 Cost: 79.678886
Epoch  200/1000 Cost: 41.509422
Epoch  300/1000 Cost: 29.618868
Epoch  400/1000 Cost: 22.692175
Epoch  500/1000 Cost: 17.169975
Epoch  600/1000 Cost: 12.759546
Epoch  700/1000 Cost: 9.192520
Epoch  800/1000 Cost: 6.385424
Epoch  900/1000 Cost: 4.327537
Epoch 1000/1000 Cost: 3.007025


In [64]:
#모델 테스트

with torch.no_grad():
    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())




Accuracy: 0.0
